<H1> InfiniBox CSI driver demo </H1>
Contact: Gregory Touretsky, @gregnsk

<H2> Install the driver using OpenShift Operator </H2>
<img src=openshift-operator.png width=50%>

<B>Notes</B>
<li>Infinidat CSI driver is supported starting from Kubernetes 1.14 (OpenShift 4.2)
<li>For earlier versions, customers may use Infinidat Dynamic Provisioner for Kubernetes (supported since Kubernetes 1.6)
<li>Kubernetes versions prior to 1.17 (OpenShift 4.4): some of CSI features like snapshots, clones, or raw block volumes might not be supported or enabled by default. Refer to relevant documentation for more details on feature-gate enablement
<li>Examples in this document use kubectl command,same commands will work with OpenShift "oc" interface
<li>Infinidat CSI driver can be installed using Helm Chart or OpenShift Operator

In [46]:
kubectl get nodes

NAME          STATUS   ROLES    AGE   VERSION
gtouret-k51   Ready    master   26d   v1.18.0
gtouret-k52   Ready    <none>   26d   v1.18.0
io307         Ready    <none>   26d   v1.18.0


In [47]:
# We recommend to install CSI driver in a dedicated namespace. Here it's running in a namespace "ibox"
# There is a single Controller (csi-infinibox-driver) instance per cluster 
# and a single Node (csi-infinibox-node) instance per worker node 
kubectl get pod -n ibox -o wide

NAME                       READY   STATUS    RESTARTS   AGE     IP              NODE          NOMINATED NODE   READINESS GATES
csi-infinibox-driver-0     5/5     Running   0          7d15h   10.244.2.55     io307         <none>           <none>
csi-infinibox-node-85g4t   2/2     Running   0          7d15h   172.20.87.214   gtouret-k51   <none>           <none>
csi-infinibox-node-jw9hx   2/2     Running   0          7d15h   172.20.87.99    gtouret-k52   <none>           <none>
csi-infinibox-node-qspr4   2/2     Running   0          7d15h   172.20.78.63    io307         <none>           <none>


<H2>Create a storage class <H2>

In [48]:
cat nfs/storageclass.yaml

apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: ibox-storageclass-demo
provisioner: infinibox-csi-driver
reclaimPolicy: Delete
volumeBindingMode: Immediate
allowVolumeExpansion: true
parameters:
    pool_name: k8s_csi
    network_space: nas1
    provision_type: THIN    
    storage_protocol: nfs   
    nfs_mount_options: hard,rsize=1048576,wsize=1048576
    nfs_export_permissions : "[{'access':'RW','client':'*','no_root_squash':true}]"
    ssd_enabled: "true"
    csi.storage.k8s.io/provisioner-secret-name: infinibox-creds
    csi.storage.k8s.io/provisioner-secret-namespace: ibox
    csi.storage.k8s.io/controller-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/controller-publish-secret-namespace: ibox
    csi.storage.k8s.io/node-stage-secret-name: infinibox-creds
    csi.storage.k8s.io/node-stage-secret-namespace: ibox
    csi.storage.k8s.io/node-publish-secret-name: infinibox-creds
    csi.storage.k8s.io/node-publish-secret-namespace: ibox
    csi.storage

In [49]:
kubectl create -f nfs/storageclass.yaml

storageclass.storage.k8s.io/ibox-storageclass-demo created


In [50]:
kubectl get sc ibox-storageclass-demo

NAME                     PROVISIONER            RECLAIMPOLICY   VOLUMEBINDINGMODE   ALLOWVOLUMEEXPANSION   AGE
ibox-storageclass-demo   infinibox-csi-driver   Delete          Immediate           true                   7s


<H2> Create a Persistent Volume Claim (PVC) </H2>

In [51]:
cat nfs/pvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-pvc-demo
  namespace: demo
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 1Gi
  storageClassName: ibox-storageclass-demo


In [52]:
kubectl create -f nfs/pvc.yaml

persistentvolumeclaim/ibox-pvc-demo created


In [53]:
kubectl get pvc ibox-pvc-demo -n demo

NAME            STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-pvc-demo   Bound    csi-376373bc1a   1Gi        RWX            ibox-storageclass-demo   5s


In [55]:
kubectl get pv csi-376373bc1a

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                STORAGECLASS             REASON   AGE
csi-376373bc1a   1Gi        RWX            Delete           Bound    demo/ibox-pvc-demo   ibox-storageclass-demo            35s


<H2> Create a snapshot class </H2>

In [56]:
cat nfs/snapshotclass.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshotClass
metadata:
  name: ibox-snapshotclass-demo
snapshotter: infinibox-csi-driver
parameters:
  csi.storage.k8s.io/snapshotter-secret-name: infinibox-creds
  csi.storage.k8s.io/snapshotter-secret-namespace: ibox



In [57]:
kubectl create -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io/ibox-snapshotclass-demo created


In [58]:
kubectl get volumesnapshotclass

NAME                      AGE
ibox-snapshotclass-demo   4s


<H2> Create a snapshot </H2>

In [59]:
cat nfs/snapshot.yaml

apiVersion: snapshot.storage.k8s.io/v1alpha1
kind: VolumeSnapshot
metadata:
  name: ibox-pvc-snapshot-demo
  namespace: demo
spec:
  snapshotClassName: ibox-snapshotclass-demo
  source:
    name: ibox-pvc-demo
    kind: PersistentVolumeClaim



In [60]:
kubectl create -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io/ibox-pvc-snapshot-demo created


In [61]:
kubectl get volumesnapshot ibox-pvc-snapshot-demo -n demo

NAME                     AGE
ibox-pvc-snapshot-demo   5s


In [62]:
kubectl describe volumesnapshot ibox-pvc-snapshot-demo  -n demo | grep snapcontent

  Snapshot Content Name:  snapcontent-f924517b-b75e-4860-b320-121c381ca60c


In [63]:
kubectl get volumesnapshotcontent snapcontent-f924517b-b75e-4860-b320-121c381ca60c

NAME                                               AGE
snapcontent-f924517b-b75e-4860-b320-121c381ca60c   27s


<H2> Create a new PVC from the snapshot (restore) </H2>

In [64]:
cat nfs/restoresnapshot.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ibox-snapshot-pvc-restore-demo-2
  namespace: demo
spec:
  storageClassName: ibox-storageclass-demo
  dataSource:
    name: ibox-pvc-snapshot-demo
    kind: VolumeSnapshot
    apiGroup: "snapshot.storage.k8s.io"
  accessModes:
    - ReadWriteMany
  resources:
    requests:
      storage: 1Gi



In [65]:
kubectl create -f nfs/restoresnapshot.yaml

persistentvolumeclaim/ibox-snapshot-pvc-restore-demo-2 created


In [66]:
kubectl get pvc ibox-snapshot-pvc-restore-demo-2 -n demo

NAME                               STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-snapshot-pvc-restore-demo-2   Bound    csi-01e10b9579   1Gi        RWX            ibox-storageclass-demo   5s


<H2> Create a new PVC from the existing PVC (clone) </H2>

In [67]:
cat nfs/clonepvc.yaml

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
    name: ibox-pvc-clone-demo
    namespace: demo
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: ibox-storageclass-demo
  resources:
    requests:
      storage: 1Gi
  dataSource:
    kind: PersistentVolumeClaim
    name: ibox-pvc-demo


In [68]:
kubectl create -f nfs/clonepvc.yaml

persistentvolumeclaim/ibox-pvc-clone-demo created


In [69]:
kubectl get pvc ibox-pvc-clone-demo -n demo

NAME                  STATUS   VOLUME           CAPACITY   ACCESS MODES   STORAGECLASS             AGE
ibox-pvc-clone-demo   Bound    csi-af218b82c3   1Gi        RWO            ibox-storageclass-demo   5s


<H2> Create a pod with a PVC attached </H2>

In [70]:
cat nfs/app.yaml

kind: Pod
apiVersion: v1
metadata:
  name: ibox-pod-pvc-demo
  namespace: demo
spec:
  containers:
    - name: my-frontend
      image: busybox
      volumeMounts:
      - mountPath: "/tmp/data"
        name: ibox-csi-volume
      command: [ "sleep", "1000" ]
  volumes:
    - name: ibox-csi-volume
      persistentVolumeClaim:
        claimName: ibox-pvc-demo


In [71]:
kubectl create -f nfs/app.yaml

pod/ibox-pod-pvc-demo created


In [72]:
kubectl get pod ibox-pod-pvc-demo -n demo

NAME                READY   STATUS    RESTARTS   AGE
ibox-pod-pvc-demo   1/1     Running   0          23s


In [73]:
kubectl exec -it ibox-pod-pvc-demo -n demo -- df -h /tmp/data

Filesystem                Size      Used Available Use% Mounted on
172.20.37.58:/csi-376373bc1a
                          1.0G      1.0M   1023.0M   0% /tmp/data


<H2>Deploying an application with Helm</H2>
helm install mydb --set mysqlRootPassword=secretpassword,mysqlUser=my-user, \<br>
mysqlPassword=my-password,mysqlDatabase=my-database,<b>persistence.storageClass=ibox-storageclass-demo</b>, \<br>
nodeSelector.protocol=iscsi stable/mysql

<H2> Clean everything </H2>

In [74]:
kubectl delete -f nfs/app.yaml

pod "ibox-pod-pvc-demo" deleted


In [75]:
kubectl delete -f nfs/clonepvc.yaml

persistentvolumeclaim "ibox-pvc-clone-demo" deleted


In [76]:
kubectl delete -f nfs/restoresnapshot.yaml

persistentvolumeclaim "ibox-snapshot-pvc-restore-demo-2" deleted


In [77]:
kubectl delete -f nfs/snapshot.yaml

volumesnapshot.snapshot.storage.k8s.io "ibox-pvc-snapshot-demo" deleted


In [78]:
kubectl delete -f nfs/snapshotclass.yaml

volumesnapshotclass.snapshot.storage.k8s.io "ibox-snapshotclass-demo" deleted


In [80]:
kubectl delete -f nfs/pvc.yaml

persistentvolumeclaim "ibox-pvc-demo" deleted


In [81]:
kubectl delete -f nfs/storageclass.yaml

storageclass.storage.k8s.io "ibox-storageclass-demo" deleted
